### Install `litellm`

This cell installs the `litellm` library, a lightweight package that provides a unified interface for interacting with various Large Language Model (LLM) APIs, including OpenAI's. It's a foundational dependency for the agent to communicate with the chosen LLM.

In [1]:
!pip install litellm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 109.9 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.76.0
    Uninstalling grpcio-1.76.0:
      Successfully uninstalled grpcio-1.76.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


### Import Necessary Modules

This cell imports all the required Python modules and classes for the agent's functionality. Key imports include:

*   `os`, `json`, `time`, `traceback`, `inspect`: For system interactions, data handling, time tracking, error reporting, and function introspection.
*   `dataclasses`, `typing`: For creating structured data classes and type hints, improving code readability and maintainability.
*   `litellm.completion`: The core function from `litellm` used to make API calls to the LLM.

In [ ]:
import os
import json
import time
import traceback
import inspect
from dataclasses import dataclass, field
from typing import get_type_hints, List, Callable, Dict, Any

from litellm import completion

### API Key Handling

This cell defines and executes the `ensure_api_key` function, which is responsible for securely obtaining and setting the `OPENAI_API_KEY`. It prioritizes environment variables and falls back to Colab's user data secrets for flexibility. This ensures that the agent can authenticate its requests to the OpenAI (or other compatible) LLM service, which is essential for its operation.

In [ ]:
# ========== API KEY HANDLING ==========

def ensure_api_key():
    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        # Optional Colab fallback
        try:
            from google.colab import userdata  # type: ignore
            api_key = userdata.get("OPENAI_API_KEY")
        except Exception:
            api_key = None

    if not api_key:
        raise RuntimeError(
            "OPENAI_API_KEY is not set. "
            "Set it in your environment or (in Colab) via the secrets UI."
        )

    os.environ["OPENAI_API_KEY"] = api_key


ensure_api_key()

### Tool Registration Layer

This cell establishes the framework for registering and managing tools (functions) that the agent can use. It includes:

*   `tools` and `tools_by_tag`: Global dictionaries to store tool metadata.
*   `get_tool_metadata`: A function that extracts crucial information (name, description, parameters, terminal status, tags) from a Python function, formatting it for LLM consumption.
*   `register_tool`: A decorator that simplifies the process of marking a Python function as an agent tool, automatically collecting its metadata and adding it to the registry.

This layer allows the agent to dynamically discover and invoke functions based on the LLM's decisions.

In [ ]:
# ========== TOOL REGISTRATION LAYER ==========

tools: Dict[str, dict] = {}
tools_by_tag: Dict[str, List[str]] = {}


def get_tool_metadata(
    func: Callable,
    tool_name: str = None,
    description: str = None,
    parameters_override: dict = None,
    terminal: bool = False,
    tags: List[str] = None,
) -> dict:
    tool_name = tool_name or func.__name__
    description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

    if parameters_override is None:
        signature = inspect.signature(func)
        type_hints = get_type_hints(func)

        args_schema = {
            "type": "object",
            "properties": {},
            "required": [],
        }

        def get_json_type(param_type):
            if param_type == str:
                return "string"
            elif param_type == int:
                return "integer"
            elif param_type == float:
                return "number"
            elif param_type == bool:
                return "boolean"
            elif param_type == list:
                return "array"
            elif param_type == dict:
                return "object"
            else:
                return "string"

        for param_name, param in signature.parameters.items():
            if param_name in ["action_context", "action_agent"]:
                continue

            param_type = type_hints.get(param_name, str)
            param_schema = {"type": get_json_type(param_type)}
            args_schema["properties"][param_name] = param_schema

            if param.default == inspect.Parameter.empty:
                args_schema["required"].append(param_name)
    else:
        args_schema = parameters_override

    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or [],
    }


def register_tool(
    tool_name: str = None,
    description: str = None,
    parameters_override: dict = None,
    terminal: bool = False,
    tags: List[str] = None,
):
    def decorator(func: Callable):
        metadata = get_tool_metadata(
            func=func,
            tool_name=tool_name,
            description=description,
            parameters_override=parameters_override,
            terminal=terminal,
            tags=tags,
        )

        tools[metadata["tool_name"]] = {
            "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"],
        }

        for tag in metadata["tags"]:
            if tag not in tools_by_tag:
                tools_by_tag[tag] = []
            tools_by_tag[tag].append(metadata["tool_name"])

        return func

    return decorator

### Prompt & LLM Wrapper

This cell defines how the agent constructs prompts for the LLM and processes its responses:

*   `Prompt` dataclass: A structured way to hold messages and tool definitions for an LLM call.
*   `to_openai_tools`: Converts the agent's `Action` objects into the format expected by OpenAI's function-calling API.
*   `generate_response`: A wrapper function that sends the constructed prompt to the `litellm` completion API. It handles both scenarios where the LLM returns a text response and when it decides to call a tool, extracting the tool name and arguments if applicable. This is the core communication mechanism with the LLM.

In [ ]:
# ========== PROMPT & LLM WRAPPER ==========

@dataclass
class Prompt:
    messages: List[Dict] = field(default_factory=list)
    tools: List[Dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)


def to_openai_tools(actions: List["Action"]) -> List[dict]:
    return [
        {
            "type": "function",
            "function": {
                "name": action.name,
                "description": action.description[:1024],
                "parameters": action.parameters,
            },
        }
        for action in actions
    ]


def generate_response(prompt: Prompt) -> dict:
    """
    Return a dict:
    {
      "tool": <tool_name or None>,
      "args": <dict or None>,
      "raw_message": <assistant content if no tool>
    }
    """
    messages = prompt.messages
    tools = prompt.tools

    if not tools:
        response = completion(
            model="openai/gpt-4o-mini",
            messages=messages,
            max_tokens=1024,
        )
        content = response.choices[0].message.content
        return {"tool": None, "args": None, "raw_message": content}

    response = completion(
        model="openai/gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=1024,
    )

    msg = response.choices[0].message
    tool_calls = getattr(msg, "tool_calls", None)

    if tool_calls:
        tool_call = tool_calls[0]
        tool_name = tool_call.function.name
        try:
            args = json.loads(tool_call.function.arguments or "{}")
        except json.JSONDecodeError:
            args = {}
        return {"tool": tool_name, "args": args, "raw_message": None}
    else:
        # No tool call – just assistant text
        return {"tool": None, "args": None, "raw_message": msg.content}

### Core Agent Primitives

This cell defines the fundamental building blocks (primitives) for the agent's architecture:

*   `Goal`: Represents an objective or task for the agent, often with a priority and description.
*   `Action`: Encapsulates a callable function (a tool), its description, parameters, and whether its execution should terminate the agent's run.
*   `ActionRegistry`: A container for `Action` objects, allowing the agent to look up and retrieve actions by name.
*   `Memory`: Stores the entire interaction history, including user inputs, agent decisions (tool calls or messages), and environment results. This provides context for the LLM.
*   `Environment`: Responsible for executing the agent's chosen `Action`s and formatting their results, including handling potential errors.

These classes work together to provide a structured way for the agent to perceive, plan, and act.

In [ ]:
# ========== CORE AGENT PRIMITIVES ==========

@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str


class Action:
    def __init__(
        self,
        name: str,
        function: Callable,
        description: str,
        parameters: Dict,
        terminal: bool = False,
    ):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        return self.function(**args)


class ActionRegistry:
    def __init__(self):
        self.actions: Dict[str, Action] = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str) -> "Action | None":
        return self.actions.get(name)

    def get_actions(self) -> List[Action]:
        return list(self.actions.values())


class Memory:
    def __init__(self):
        self.items: List[Dict] = []

    def add_memory(self, memory: dict):
        self.items.append(memory)

    def get_memories(self, limit: int = None) -> List[Dict]:
        return self.items if limit is None else self.items[:limit]

    def copy_without_system_memories(self) -> "Memory":
        filtered_items = [m for m in self.items if m.get("type") != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory


class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc(),
            }

    def format_result(self, result: Any) -> dict:
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z"),
        }

### Agent Language (Function Calling)

This cell defines the `AgentLanguage` abstraction and its concrete implementation, `AgentFunctionCallingActionLanguage`. This class dictates how the agent's internal state (goals, memory, available actions) is translated into a prompt that the LLM can understand, particularly for models supporting function calling:

*   `format_goals`: Converts `Goal` objects into system messages for the LLM.
*   `format_memory`: Translates the `Memory` log into a sequence of user and assistant messages.
*   `construct_prompt`: Assembles the full prompt, including goals, memory, and tool definitions, for the LLM.
*   `parse_response`: Interprets the LLM's raw response, extracting whether a tool was called and its arguments, or if it was a plain text message.

In [ ]:
# ========== AGENT LANGUAGE (FUNCTION CALLING) ==========

class AgentLanguage:
    def construct_prompt(
        self,
        actions: List[Action],
        environment: Environment,
        goals: List[Goal],
        memory: Memory,
    ) -> Prompt:
        raise NotImplementedError

    def parse_response(self, response: dict) -> dict:
        raise NotImplementedError


class AgentFunctionCallingActionLanguage(AgentLanguage):
    def format_goals(self, goals: List[Goal]) -> List[Dict]:
        sep = "\n-------------------\n"
        goal_instructions = "\n\n".join(
            [f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals]
        )
        return [{"role": "system", "content": goal_instructions}]

    def format_memory(self, memory: Memory) -> List[Dict]:
        items = memory.get_memories()
        mapped = []
        for item in items:
            content = item.get("content")
            if content is None:
                content = json.dumps(item, indent=4)

            type_ = item.get("type")
            if type_ in ("assistant", "environment"):
                mapped.append({"role": "assistant", "content": content})
            else:
                mapped.append({"role": "user", "content": content})
        return mapped

    def construct_prompt(
        self,
        actions: List[Action],
        environment: Environment,
        goals: List[Goal],
        memory: Memory,
    ) -> Prompt:
        messages = []
        messages += self.format_goals(goals)
        messages += self.format_memory(memory)
        tools_spec = to_openai_tools(actions)
        return Prompt(messages=messages, tools=tools_spec)

    def parse_response(self, response: dict) -> dict:
        # response already has tool / args / raw_message
        return response

### Python Action Registry

This cell introduces `PythonActionRegistry`, a specialized `ActionRegistry` that automatically populates itself with tools registered using the `@register_tool` decorator. It can be configured to filter tools by tags or specific names. Crucially, it also handles the registration of a special `terminate` tool, which is used to signal the completion of the agent's task and return a final output.

In [ ]:
# ========== PYTHON ACTION REGISTRY ==========

class PythonActionRegistry(ActionRegistry):
    def __init__(self, tags: List[str] = None, tool_names: List[str] = None):
        super().__init__()
        self.terminate_tool_def = None

        for tool_name, tool_desc in tools.items():
            if tool_name == "terminate":
                self.terminate_tool_def = tool_desc

            if tool_names and tool_name not in tool_names:
                continue

            tool_tags = tool_desc.get("tags", [])
            if tags and not any(tag in tool_tags for tag in tags):
                continue

            self.register(
                Action(
                    name=tool_name,
                    function=tool_desc["function"],
                    description=tool_desc["description"],
                    parameters=tool_desc.get("parameters", {}),
                    terminal=tool_desc.get("terminal", False),
                )
            )

    def register_terminate_tool(self):
        if self.terminate_tool_def:
            self.register(
                Action(
                    name="terminate",
                    function=self.terminate_tool_def["function"],
                    description=self.terminate_tool_def["description"],
                    parameters=self.terminate_tool_def.get("parameters", {}),
                    terminal=self.terminate_tool_def.get("terminal", False),
                )
            )
        else:
            raise RuntimeError("Terminate tool not found in tool registry")

### Agent

This cell defines the central `Agent` class, which orchestrates the entire reasoning and execution loop:

*   **Initialization**: Takes `goals`, an `agent_language` instance, an `action_registry`, an LLM `generate_response_fn`, and an `environment`.
*   `construct_prompt`: Uses the agent language to build the prompt for the LLM.
*   `get_action`: Parses the LLM's response to identify tool calls or direct text.
*   `should_terminate`: Checks if the executed action signals termination.
*   `set_current_task`: Adds the user's initial query to memory.
*   `run`: The main loop where the agent repeatedly:
    1.  Constructs a prompt based on current memory and goals.
    2.  Calls the LLM (`generate_response_fn`).
    3.  Parses the LLM's decision (tool call or text).
    4.  If a tool is called, executes it via the `environment`.
    5.  Updates its `memory` with the LLM's response and the action's result.
    6.  Checks for termination conditions.

This class embodies the agent's ability to reason, act, and learn from its interactions.

In [ ]:
# ========== AGENT ==========

class Agent:
    def __init__(
        self,
        goals: List[Goal],
        agent_language: AgentLanguage,
        action_registry: ActionRegistry,
        generate_response_fn: Callable[[Prompt], dict],
        environment: Environment,
    ):
        self.goals = goals
        self.agent_language = agent_language
        self.actions = action_registry
        self.generate_response_fn = generate_response_fn
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory,
        )

    def get_action(self, response_dict: dict):
        invocation = self.agent_language.parse_response(response_dict)
        tool_name = invocation.get("tool")
        if tool_name is None:
            return None, invocation
        action = self.actions.get_action(tool_name)
        if action is None:
            raise RuntimeError(f"Unknown tool requested by model: {tool_name}")
        return action, invocation

    def should_terminate(self, action: Action) -> bool:
        return action.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def run(self, user_input: str, memory: Memory = None, max_iterations: int = 50) -> Memory:
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            prompt = self.construct_prompt(self.goals, memory, self.actions)
            print("Agent thinking...")
            response_dict = self.generate_response_fn(prompt)
            print(f"Agent decision: {response_dict}")

            action, invocation = self.get_action(response_dict)

            # Case 1: no tool called, just assistant text – add to memory and continue
            if action is None:
                raw_msg = invocation.get("raw_message")
                if raw_msg:
                    memory.add_memory({"type": "assistant", "content": raw_msg})
                continue

            # Case 2: real tool call
            args = invocation.get("args", {}) or {}
            result = self.environment.execute_action(action, args)
            print(f"Action result: {result}")

            memory.add_memory(
                {"type": "assistant", "content": json.dumps(response_dict, indent=2)}
            )
            memory.add_memory(
                {"type": "environment", "content": json.dumps(result, indent=2)}
            )

            if self.should_terminate(action):
                break

        return memory

### Tools (File Operations + Terminate)

This cell defines the concrete tools that the agent can invoke, using the `@register_tool` decorator:

*   `read_project_file(name: str)`: Reads and returns the content of a specified file. Tagged for `file_operations` and `read`.
*   `list_project_files()`: Lists all Python files in the current directory. Tagged for `file_operations` and `list`.
*   `terminate(message: str)`: A special terminal tool. When called, it signals the agent to stop execution and passes a final message (e.g., a complete README). Tagged for `system` and marked as `terminal=True`.

In [ ]:
# ========== TOOLS (FILE OPS + TERMINATE) ==========

@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file."""
    with open(name, "r", encoding="utf-8") as f:
        return f.read()


@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current project directory."""
    return sorted([file for file in os.listdir(".") if file.endswith(".py")])


@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message.

    The `message` should contain the complete README in markdown format.
    """
    return f"{message}\nTerminating..."

### Goals & Agent Wiring

This cell configures the specific goals and wires together all the components of the agent for this particular task:

*   `goals`: A list of `Goal` objects outlining the agent's objectives: first, to gather project information, and then to write a README and terminate.
*   `action_registry`: An instance of `PythonActionRegistry` is created, configured to include tools tagged with `file_operations` and `system` (ensuring file reading/listing and the `terminate` tool are available). The `terminate` tool is explicitly registered.
*   `agent`: The main `Agent` instance is created, bringing together the defined `goals`, `agent_language` (`AgentFunctionCallingActionLanguage`), configured `action_registry`, the LLM `generate_response_fn`, and the `environment`. This setup effectively defines what the agent will try to achieve and how it will interact with its surroundings.

In [ ]:
# ========== GOALS & AGENT WIRING ==========

goals = [
    Goal(
        priority=1,
        name="Gather Information",
        description=(
            "Use the tools `list_project_files` and `read_project_file` to read ALL relevant "
            "Python files in the project. Build an understanding of the project's purpose, "
            "entrypoint, main components, and how they interact."
        ),
    ),
    Goal(
        priority=1,
        name="Write README and Terminate",
        description=(
            "After you have read and understood the project files, call the `terminate` tool EXACTLY ONCE. "
            "In the `message` parameter, pass a COMPLETE README in valid Markdown. The README should include:\n"
            "- Project title\n"
            "- Short description\n"
            "- How it works (overview of main.py, utils.py, service.py etc.)\n"
            "- How to run the project\n"
            "- Example usage (what the script prints)\n"
            "Do NOT output the README as plain text; ALWAYS return it via the terminate tool."
        ),
    ),
]

action_registry = PythonActionRegistry(tags=["file_operations", "system"])
action_registry.register_terminate_tool()

agent = Agent(
    goals=goals,
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=action_registry,
    generate_response_fn=generate_response,
    environment=Environment(),
)

### Run Agent

This is the main execution block that initiates the agent's operation. It performs the following:

1.  Sets `user_input`: Defines the initial prompt or task given to the agent, which is to "Write a README for this project."
2.  Calls `agent.run(user_input)`: Starts the agent's iterative process of thinking, acting, and learning based on its goals and available tools.
3.  Prints `final_memory`: After the agent has completed its execution (either by terminating or reaching `max_iterations`), this displays the complete log of the agent's interactions, including all user prompts, LLM decisions, and environment results.

In [6]:
# ========== RUN ==========

if __name__ == "__main__":
    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)
    print("\n===== FINAL MEMORY =====")
    for m in final_memory.get_memories():
        print(m)


Agent thinking...
Agent decision: {'tool': 'list_project_files', 'args': {}, 'raw_message': None}
Action result: {'tool_executed': True, 'result': ['main.py', 'service.py', 'utils.py'], 'timestamp': '2025-11-30T11:58:22+0000'}
Agent thinking...
Agent decision: {'tool': 'read_project_file', 'args': {'name': 'main.py'}, 'raw_message': None}
Action result: {'tool_executed': True, 'result': 'from utils import add_numbers\nfrom service import greet_user\n\ndef main():\n    print(greet_user("Vansh"))\n    result = add_numbers(10, 20)\n    print(f"Sum: {result}")\n\nif __name__ == "__main__":\n    main()\n', 'timestamp': '2025-11-30T11:58:23+0000'}
Agent thinking...
Agent decision: {'tool': 'read_project_file', 'args': {'name': 'service.py'}, 'raw_message': None}
Action result: {'tool_executed': True, 'result': 'def greet_user(name: str) -> str:\n    """\n    Returns a welcome message for the user.\n    """\n    return f"Hello, {name}! Welcome to the sample project."\n', 'timestamp': '2025-11